In [4]:
#!pip3 install summa
#!pip install python-rake
import pandas as pd
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
from pymorphy2.tokenizers import simple_word_tokenize
from string import punctuation
punctuation = punctuation + '—–«»·№˚¼°'
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from summa import keywords
import RAKE
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Zlata\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Источник
Данные были взяты взяты из научного журнала «Постнаука» из следующих статей:

https://postnauka.ru/faq/155130

https://postnauka.ru/longreads/156624

https://postnauka.ru/talks/156733

https://postnauka.ru/longreads/156663

https://postnauka.ru/longreads/156517

Ключевые слова представлены в виде тегов к статьям

In [5]:
df = pd.read_csv('data.csv', error_bad_lines=False)

In [6]:
df

,text,keywords,my_keywords
0,Психолог Марк Ламли о нарушении деятельности ч...,"биология, психология, болезнь, боль, болезни, ...","боль, генетика, хроническая боль, тревожность,..."
1,В привычном понимании термин «психосоматика» с...,"психология, кардиология, психотерапия, медицин...","психосоматика, болезнь, расстройства, гипертон..."
2,"Обучение происходит продуктивнее, если подходы...","психология, мотивация, обучение, обучение с уч...","обучение, мотивация, навыки, умения, мышление,..."
3,Мирный атом — это одна из основных надежд чело...,"атом, топливо, ядерная физика, топливные элеме...","атом, ядерная энергетика, ядерное топливо, ура..."
4,В последние годы интерес к правильному питанию...,"медицина, белки, питание, еда, калории, углево...","питание, медицина, диета, белки, жиры, углевод..."


In [7]:
def preprocessing(text):
    text = text.lower()
    for i in text:
        if i in punctuation:
            text = text.replace(i, '')
    text = text.replace('\n', ' ')
    text = text.replace('       ', ' ')
    text = text.replace('   ', ' ')
    text = text.replace('  ', ' ')
    text = text.replace('\ufeff', '')
    return text

In [8]:
def normalize_text(text):
    lemmas = []
    for t in simple_word_tokenize(text):
        lemmas.append(
            morph.parse(t)[0].normal_form
        )
    return ' '.join(lemmas)

In [9]:
df['text'] = df['text'].apply(preprocessing)

In [10]:
tokens_num = []
for i in df['text']:
    tokens_num.append(len(i.split(' ')))
df['tokens_num'] = tokens_num

In [11]:
df

,text,keywords,my_keywords,tokens_num
0,психолог марк ламли о нарушении деятельности ч...,"биология, психология, болезнь, боль, болезни, ...","боль, генетика, хроническая боль, тревожность,...",2689
1,в привычном понимании термин психосоматика ста...,"психология, кардиология, психотерапия, медицин...","психосоматика, болезнь, расстройства, гипертон...",460
2,обучение происходит продуктивнее если подходы ...,"психология, мотивация, обучение, обучение с уч...","обучение, мотивация, навыки, умения, мышление,...",1836
3,мирный атом это одна из основных надежд челове...,"атом, топливо, ядерная физика, топливные элеме...","атом, ядерная энергетика, ядерное топливо, ура...",2840
4,в последние годы интерес к правильному питанию...,"медицина, белки, питание, еда, калории, углево...","питание, медицина, диета, белки, жиры, углевод...",1977


## Эталонные ключевые слова

In [12]:
keywords_intersection = [list(set(df['keywords'][i].split(', ')) & set(df['my_keywords'][i].split(', '))) for i in range(len(df))]
keywords_intersection

[['боль'],
 ['психосоматика'],
 ['мотивация', 'обучение'],
 ['атом', 'уран'],
 ['питание', 'медицина', 'калории', 'белки', 'углеводы']]

Кажется, что пересечение ключевых слов довольно маленькое и стоит использовать их объединение

In [13]:
keywords_combining = [list(set(df['keywords'][i].split(', ')) | set(df['my_keywords'][i].split(', '))) for i in range(len(df))]
keywords_combining

[['хроническая боль',
  'болезни',
  'психология',
  'лечение',
  'биология',
  'журнал',
  'эмоции',
  'боль',
  'тревожность',
  'болезнь',
  'естественные науки',
  'генетика'],
 ['медицинская диагностика',
  'биопсихосоциальный подход',
  'психология',
  'психотерапия',
  'журнал',
  'терапия',
  'кардиология',
  'расстройства',
  'лекарства',
  'гипертония',
  'когнитивные науки',
  'болезнь',
  'психосоматика',
  'диагноз'],
 ['психология',
  'журнал',
  'навыки',
  'обучение с учителем',
  'обучение',
  'социальные науки',
  'мотивация',
  'взрослые',
  'когнитивные способности',
  'мышление',
  'дети',
  'умения'],
 ['журнал',
  'топливо',
  'отходы',
  'атом',
  'нейтроны',
  'толерантное топливо',
  'топливные элементы',
  'уран',
  'ядерное топливо',
  'ядерная энергетика',
  'ядерная физика'],
 ['витамины',
  'журнал',
  'медицина',
  'питание',
  'диета',
  'калории',
  'естественные науки',
  'белки',
  'углеводы',
  'еда',
  'жиры']]

In [14]:
df['best_keywords'] = keywords_combining

## Rake

In [15]:
stop = stopwords.words('russian')
stop.extend(('это', 'также', 'весь', 'который', 'изза', 'нам', 'которые', 'изз'))
rake = RAKE.Rake(stop)

In [16]:
def rake_keywords(text):
    text_keywords = rake.run(normalize_text(text), maxWords=3, minFrequency=2)
    return text_keywords

In [17]:
rake_keywords_list = []
for text in df['text']:
    rake_keywords_list.append(rake_keywords(text))

In [18]:
rake_keywords_list

[[('минимум шесть месяц', 8.666666666666666),
  ('нейронный путь задействовать', 8.0),
  ('хронический боль', 5.006349206349206),
  ('свой тело', 4.375),
  ('вызывать боль', 4.3730158730158735),
  ('субъективный переживание', 4.333333333333334),
  ('нормальный жизнь', 4.333333333333334),
  ('генерирование эмоция', 3.8),
  ('человек', 2.3636363636363638),
  ('боль', 2.2063492063492065),
  ('травма', 2.1666666666666665),
  ('некоторый', 2.142857142857143),
  ('пациент', 2.1176470588235294),
  ('мочь', 2.0),
  ('связать', 2.0),
  ('лечение', 2.0),
  ('категория', 2.0),
  ('тело', 2.0),
  ('подход', 2.0),
  ('проблема', 1.9166666666666667),
  ('рассматривать', 1.8),
  ('мозг', 1.8),
  ('случай', 1.8),
  ('говорить', 1.75),
  ('работа', 1.75),
  ('расстройство', 1.75),
  ('фибромиалгия', 1.6666666666666667),
  ('работать', 1.6666666666666667),
  ('стресс', 1.6666666666666667),
  ('наблюдаться', 1.6666666666666667),
  ('влиять', 1.6),
  ('сигнал', 1.6),
  ('заживать', 1.3333333333333333),
  

In [19]:
df['rake_keywords'] = rake_keywords_list
df['rake_keywords'] = df['rake_keywords'].apply(lambda x: [i[0] for i in x])

## TextRank

In [20]:
def textrank_keywords(text):
    text_keywords = keywords.keywords(normalize_text(text), language='russian', additional_stopwords=stop, scores=True)
    return text_keywords

In [21]:
textrank_keywords_list = []
for text in df['text']:
    textrank_keywords_list.append(textrank_keywords(text))

In [22]:
textrank_keywords_list

[[('боль', 0.6030416301637213),
  ('пациент', 0.1905855829023084),
  ('мочь', 0.14137519539774396),
  ('деятельность человек', 0.13487915780345666),
  ('хронический', 0.12141572915234473),
  ('эмоциональный', 0.10906228795029209),
  ('некоторый', 0.1008390757939825),
  ('свой', 0.09349248192499998),
  ('часто', 0.0913555657127427),
  ('часть', 0.0913555657127427),
  ('частью', 0.0913555657127427),
  ('определённый проблема', 0.091023186913409),
  ('хороший', 0.09023223906975586),
  ('травма', 0.08855067277200193),
  ('связать', 0.08667773590122872),
  ('связанный', 0.08667773590122872),
  ('лечение', 0.08450620675444531),
  ('болевой ощущение', 0.0844411619314868),
  ('заболевание', 0.08433527405997002),
  ('имя', 0.08095365860355863),
  ('конкретный', 0.076228698257633),
  ('конкретно', 0.076228698257633),
  ('необходимый', 0.07437649524297302),
  ('разный', 0.07157833543045766),
  ('нужно', 0.0712523121460483),
  ('нужный', 0.0712523121460483),
  ('расстройство', 0.07093839638818303)

In [23]:
df['textrank_keywords'] = textrank_keywords_list
df['textrank_keywords'] = df['textrank_keywords'].apply(lambda x: [i[0] for i in x])

## TF-IDF

In [24]:
vectorizer = TfidfVectorizer(stop_words=stop)

In [25]:
def tfidf_keywords(text):
    text = normalize_text(text)
    X = vectorizer.fit_transform([text])
    feature_names = np.array(vectorizer.get_feature_names())
    denselist = np.array(X.todense())
    indices = denselist.argsort()[0]
    text_keywords = list(feature_names[indices[::-1][:15]])
    return text_keywords

In [26]:
tfidf_keywords_list = []
for text in df['text']:
    tfidf_keywords_list.append(tfidf_keywords(text))

In [27]:
tfidf_keywords_list

[['боль',
  'хронический',
  'человек',
  'пациент',
  'болевой',
  'мочь',
  'связать',
  'проблема',
  'некоторый',
  'мозг',
  'ощущение',
  'лечение',
  'хороший',
  'свой',
  'расстройство'],
 ['заболевание',
  'приводить',
  'расстройство',
  'человек',
  'соматический',
  'гипертония',
  'фактор',
  'случай',
  'болезнь',
  'психосоматический',
  'проблема',
  'стресс',
  'психологический',
  'развитие',
  'год'],
 ['обучение',
  'ребёнок',
  'действие',
  'когнитивный',
  'способ',
  'мочь',
  'способность',
  'взрослый',
  'хороший',
  'собственный',
  'процесс',
  'этап',
  'развитие',
  'мышление',
  'знание'],
 ['топливо',
  'реактор',
  'ядерный',
  'уран',
  'плутоний',
  'нейтрон',
  'однако',
  'год',
  'деление',
  'энергетика',
  'энергия',
  'материал',
  'делиться',
  'большой',
  'ядро'],
 ['углевод',
  'жир',
  'организм',
  'человек',
  'энергия',
  'продукт',
  'калория',
  'количество',
  'белок',
  'грамм',
  'пища',
  'должный',
  'простой',
  'белка',
  'нап

In [28]:
df['tfidf_keywords'] = tfidf_keywords_list

In [29]:
df

,text,keywords,my_keywords,tokens_num,best_keywords,rake_keywords,textrank_keywords,tfidf_keywords
0,психолог марк ламли о нарушении деятельности ч...,"биология, психология, болезнь, боль, болезни, ...","боль, генетика, хроническая боль, тревожность,...",2689,"[хроническая боль, болезни, психология, лечени...","[минимум шесть месяц, нейронный путь задейство...","[боль, пациент, мочь, деятельность человек, хр...","[боль, хронический, человек, пациент, болевой,..."
1,в привычном понимании термин психосоматика ста...,"психология, кардиология, психотерапия, медицин...","психосоматика, болезнь, расстройства, гипертон...",460,"[медицинская диагностика, биопсихосоциальный п...",[гипертония],"[фактор, человек, расстройство, соматический з...","[заболевание, приводить, расстройство, человек..."
2,обучение происходит продуктивнее если подходы ...,"психология, мотивация, обучение, обучение с уч...","обучение, мотивация, навыки, умения, мышление,...",1836,"[психология, журнал, навыки, обучение с учител...","[способ обучение, ребёнок, способ, мочь, важны...","[ребёнок, взрослый, собственный, действие, раз...","[обучение, ребёнок, действие, когнитивный, спо..."
3,мирный атом это одна из основных надежд челове...,"атом, топливо, ядерная физика, топливные элеме...","атом, ядерная энергетика, ядерное топливо, ура...",2840,"[журнал, топливо, отходы, атом, нейтроны, толе...","[поддержание цепной реакция, быстрый нейтрон, ...","[ядерный топливо, уран, урана, плутоний, плуто...","[топливо, реактор, ядерный, уран, плутоний, не..."
4,в последние годы интерес к правильному питанию...,"медицина, белки, питание, еда, калории, углево...","питание, медицина, диета, белки, жиры, углевод...",1977,"[витамины, журнал, медицина, питание, диета, к...","[быстрый углевод, углевод, человек, организм, ...","[углевод, человек, количество, организм, проду...","[углевод, жир, организм, человек, энергия, про..."


## Морфологические/синтаксические шаблоны для ключевых слов и фраз
Возьмем следующие шаблоны ключевых слов и фраз:

In [30]:
templates = [['NOUN'], ['ADJF'], ['ADJF', 'NOUN']]

In [31]:
def templates_keywords(keywords):
    temp_keywords = []
    for keyword in keywords:
        keyword_pos = []
        for word in keyword.split(' '):
            keyword_pos.append(morph.parse(word)[0].tag.POS)
        if keyword_pos in templates:
            temp_keywords.append(keyword)
    return temp_keywords

In [32]:
templates_rake_keywords = []
for j in df['rake_keywords']:
    templates_rake_keywords.append(templates_keywords(j))
df['templates_rake_keywords'] = templates_rake_keywords

In [33]:
templates_textrank_keywords = []
for j in df['textrank_keywords']:
    templates_textrank_keywords.append(templates_keywords(j))
df['templates_textrank_keywords'] = templates_textrank_keywords

In [34]:
templates_tfidf_keywords = []
for j in df['tfidf_keywords']:
    templates_tfidf_keywords.append(templates_keywords(j))
df['templates_tfidf_keywords'] = templates_tfidf_keywords

In [35]:
df

,text,keywords,my_keywords,tokens_num,best_keywords,rake_keywords,textrank_keywords,tfidf_keywords,templates_rake_keywords,templates_textrank_keywords,templates_tfidf_keywords
0,психолог марк ламли о нарушении деятельности ч...,"биология, психология, болезнь, боль, болезни, ...","боль, генетика, хроническая боль, тревожность,...",2689,"[хроническая боль, болезни, психология, лечени...","[минимум шесть месяц, нейронный путь задейство...","[боль, пациент, мочь, деятельность человек, хр...","[боль, хронический, человек, пациент, болевой,...","[хронический боль, свой тело, субъективный пер...","[боль, пациент, мочь, хронический, эмоциональн...","[боль, хронический, человек, пациент, болевой,..."
1,в привычном понимании термин психосоматика ста...,"психология, кардиология, психотерапия, медицин...","психосоматика, болезнь, расстройства, гипертон...",460,"[медицинская диагностика, биопсихосоциальный п...",[гипертония],"[фактор, человек, расстройство, соматический з...","[заболевание, приводить, расстройство, человек...",[гипертония],"[фактор, человек, расстройство, соматический з...","[заболевание, расстройство, человек, соматичес..."
2,обучение происходит продуктивнее если подходы ...,"психология, мотивация, обучение, обучение с уч...","обучение, мотивация, навыки, умения, мышление,...",1836,"[психология, журнал, навыки, обучение с учител...","[способ обучение, ребёнок, способ, мочь, важны...","[ребёнок, взрослый, собственный, действие, раз...","[обучение, ребёнок, действие, когнитивный, спо...","[ребёнок, способ, мочь, важный, подход, этап, ...","[ребёнок, взрослый, собственный, действие, раз...","[обучение, ребёнок, действие, когнитивный, спо..."
3,мирный атом это одна из основных надежд челове...,"атом, топливо, ядерная физика, топливные элеме...","атом, ядерная энергетика, ядерное топливо, ура...",2840,"[журнал, топливо, отходы, атом, нейтроны, толе...","[поддержание цепной реакция, быстрый нейтрон, ...","[ядерный топливо, уран, урана, плутоний, плуто...","[топливо, реактор, ядерный, уран, плутоний, не...","[быстрый нейтрон, атомный энергетика, природны...","[ядерный топливо, уран, урана, плутоний, плуто...","[топливо, реактор, ядерный, уран, плутоний, не..."
4,в последние годы интерес к правильному питанию...,"медицина, белки, питание, еда, калории, углево...","питание, медицина, диета, белки, жиры, углевод...",1977,"[витамины, журнал, медицина, питание, диета, к...","[быстрый углевод, углевод, человек, организм, ...","[углевод, человек, количество, организм, проду...","[углевод, жир, организм, человек, энергия, про...","[быстрый углевод, углевод, человек, организм, ...","[углевод, человек, количество, организм, проду...","[углевод, жир, организм, человек, энергия, про..."


 ## Оценка точности, полноты, F-меры 

In [36]:
df['best_keywords'] = df['best_keywords'].apply(lambda x: [preprocessing(i) for i in x])
df['best_keywords'] = df['best_keywords'].apply(lambda x: [normalize_text(i) for i in x])
df['best_keywords']

0    [хронический боль, болезнь, психология, лечени...
1    [медицинский диагностика, биопсихосоциальный п...
2    [психология, журнал, навык, обучение с учитель...
3    [журнал, топливо, отход, атом, нейтрон, толера...
4    [витамин, журнал, медицина, питание, диета, ка...
Name: best_keywords, dtype: object

Применение шаблона к эталонным ключевым словам

In [37]:
templates_best_keywords = []
for j in df['best_keywords']:
    templates_best_keywords.append(templates_keywords(j))
df['templates_best_keywords'] = templates_best_keywords

In [38]:
df

,text,keywords,my_keywords,tokens_num,best_keywords,rake_keywords,textrank_keywords,tfidf_keywords,templates_rake_keywords,templates_textrank_keywords,templates_tfidf_keywords,templates_best_keywords
0,психолог марк ламли о нарушении деятельности ч...,"биология, психология, болезнь, боль, болезни, ...","боль, генетика, хроническая боль, тревожность,...",2689,"[хронический боль, болезнь, психология, лечени...","[минимум шесть месяц, нейронный путь задейство...","[боль, пациент, мочь, деятельность человек, хр...","[боль, хронический, человек, пациент, болевой,...","[хронический боль, свой тело, субъективный пер...","[боль, пациент, мочь, хронический, эмоциональн...","[боль, хронический, человек, пациент, болевой,...","[хронический боль, болезнь, психология, лечени..."
1,в привычном понимании термин психосоматика ста...,"психология, кардиология, психотерапия, медицин...","психосоматика, болезнь, расстройства, гипертон...",460,"[медицинский диагностика, биопсихосоциальный п...",[гипертония],"[фактор, человек, расстройство, соматический з...","[заболевание, приводить, расстройство, человек...",[гипертония],"[фактор, человек, расстройство, соматический з...","[заболевание, расстройство, человек, соматичес...","[медицинский диагностика, биопсихосоциальный п..."
2,обучение происходит продуктивнее если подходы ...,"психология, мотивация, обучение, обучение с уч...","обучение, мотивация, навыки, умения, мышление,...",1836,"[психология, журнал, навык, обучение с учитель...","[способ обучение, ребёнок, способ, мочь, важны...","[ребёнок, взрослый, собственный, действие, раз...","[обучение, ребёнок, действие, когнитивный, спо...","[ребёнок, способ, мочь, важный, подход, этап, ...","[ребёнок, взрослый, собственный, действие, раз...","[обучение, ребёнок, действие, когнитивный, спо...","[психология, журнал, навык, обучение, социальн..."
3,мирный атом это одна из основных надежд челове...,"атом, топливо, ядерная физика, топливные элеме...","атом, ядерная энергетика, ядерное топливо, ура...",2840,"[журнал, топливо, отход, атом, нейтрон, толера...","[поддержание цепной реакция, быстрый нейтрон, ...","[ядерный топливо, уран, урана, плутоний, плуто...","[топливо, реактор, ядерный, уран, плутоний, не...","[быстрый нейтрон, атомный энергетика, природны...","[ядерный топливо, уран, урана, плутоний, плуто...","[топливо, реактор, ядерный, уран, плутоний, не...","[журнал, топливо, отход, атом, нейтрон, толера..."
4,в последние годы интерес к правильному питанию...,"медицина, белки, питание, еда, калории, углево...","питание, медицина, диета, белки, жиры, углевод...",1977,"[витамин, журнал, медицина, питание, диета, ка...","[быстрый углевод, углевод, человек, организм, ...","[углевод, человек, количество, организм, проду...","[углевод, жир, организм, человек, энергия, про...","[быстрый углевод, углевод, человек, организм, ...","[углевод, человек, количество, организм, проду...","[углевод, жир, организм, человек, энергия, про...","[витамин, журнал, медицина, питание, диета, ка..."


In [39]:
def precision(best_keywords, keywords):
    precision_values = []
    for i, text in enumerate(best_keywords):
        precision_values.append(len(set(text).intersection(keywords[i]))/len(text))
    return np.array(precision_values).mean()

In [41]:
def recall(best_keywords, keywords):
    precision_values = []
    for i, text in enumerate(best_keywords):
        precision_values.append(len(set(text).intersection(keywords[i]))/len(keywords[i]))
    return np.array(precision_values).mean()

In [53]:
def fscore(best_keywords, keywords):
    precision_value = precision(best_keywords, keywords)
    recall_value = recall(best_keywords, keywords)
    return (2 * precision_value * recall_value) / (precision_value + recall_value)

## Precision

In [ ]:
precision_values = []

Precision для ключевых слов без шаблонов

In [61]:
precision_values.append(precision(df['best_keywords'], df['rake_keywords']))
precision_values.append(precision(df['best_keywords'], df['textrank_keywords']))
precision_values.append(precision(df['best_keywords'], df['tfidf_keywords']))

Precision для ключевых слов с шаблононами

In [62]:
precision_values.append(precision(df['templates_best_keywords'], df['templates_rake_keywords']))
precision_values.append(precision(df['templates_best_keywords'], df['templates_textrank_keywords']))
precision_values.append(precision(df['templates_best_keywords'], df['templates_tfidf_keywords']))

## Recall

In [ ]:
recall_values = []

Recall для ключевых слов без шаблонов

In [66]:
recall_values.append(recall(df['best_keywords'], df['rake_keywords']))
recall_values.append(recall(df['best_keywords'], df['textrank_keywords']))
recall_values.append(recall(df['best_keywords'], df['tfidf_keywords']))

Recall для ключевых слов с шаблононами

In [68]:
recall_values.append(recall(df['templates_best_keywords'], df['templates_rake_keywords']))
recall_values.append(recall(df['templates_best_keywords'], df['templates_textrank_keywords']))
recall_values.append(recall(df['templates_best_keywords'], df['templates_tfidf_keywords']))

## F-мера

In [71]:
fscore_values = []

F-мера для ключевых слов без шаблонов

In [72]:
fscore_values.append(fscore(df['best_keywords'], df['rake_keywords']))
fscore_values.append(fscore(df['best_keywords'], df['textrank_keywords']))
fscore_values.append(fscore(df['best_keywords'], df['tfidf_keywords']))

F-мера для ключевых слов с шаблонами

In [73]:
fscore_values.append(fscore(df['templates_best_keywords'], df['templates_rake_keywords']))
fscore_values.append(fscore(df['templates_best_keywords'], df['templates_textrank_keywords']))
fscore_values.append(fscore(df['templates_best_keywords'], df['templates_tfidf_keywords']))

## Общие результаты

In [82]:
df_scores = pd.DataFrame()
df_scores['method'] = ['rake', 'textrank', 'tfidf', 'rake_pos', 'textrank_pos', 'tfidf_pos']
df_scores['precision'] = precision_values
df_scores['recall'] = recall_values
df_scores['fscore'] = fscore_values

In [83]:
df_scores

,method,precision,recall,fscore
0,rake,0.135498,0.253720,0.176654
1,textrank,0.326407,0.034831,0.062946
2,tfidf,0.270130,0.213333,0.238395
3,rake_pos,0.137013,0.281172,0.184245
4,textrank_pos,0.335498,0.057221,0.097767
5,tfidf_pos,0.276190,0.228059,0.249827


## Аналитика

Во всех текстах кроме второго было выделено довольно много ключевых слов. Я думаю это связано с объемом статьи, так как остальные тексты состояли из ~2000 токенов, а вторая статья - всего из ~450. Также она написана на довольно широкую тему, а в тексте используется множество синонимов.

Можно заметить, что шаблоны все-таки оказали небольшое влияние на показатели точность в лучшую сторону. Кажется, шаблоны подобраны неплохо. В качестве тегов для статей обычно используются именное такие конструкции.

Самым точным методом оказался TF-IDF, думаю, это связано с тем, что слова, выбранные редакторами в качестве тегов, также были определены по частотности в тексте. Но его проблема в том, что в некоторых статьях для основного ключевого слова используется множество разных синонимов, а для второстепенных слов – нет, в связи с чем встрчается это слово очень часто. По результатам TextRank видно, что мы слова выделяются правильно, но находится таких слов мало (высокий относительно других precision, но низкий recall, что пагубно сказывается на общих результатах). Работает он тоже неплохо, но все-таки хуже чем TF-IDF. Rake же наоборот – находит много, но не очень точно (высокий относительно других recall, но низкий precision, но общий результат получше, чем у TextRank). Кажется, что TF-IDF – некая "золотая середина", где-то похуже других, но в целом – дает лучший результат.

Думаю, чтобы улучшить работу, можно попробовать другие шаблоны (например с краткими прилагательными или еще чем-то, но точно не с глаголами). Можно сделать условие на параметр, отвечающий за встречаемость слова в тексте, в зависимости от длины данного текста. Можно поработать с другими данными, желательно из одной конкреной сферы, но данные брать из одного типа источника. Было бы полезно поработать не с конкретными формами, а с синонимами и вообще учитывать расстояние.